In [1]:
%matplotlib inline
%load_ext autoreload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import time
from datetime import datetime
import scipy.optimize as opt
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
plt.style.use("ggplot")
mpl.pyplot.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams["font.size"] = 13
#mpl.rcParams["font.family"] = 'Osaka'

import sys
sys.path.append('../code/')

from simulation_hash_price import *
from simulation_fixed_path import make_stats
from hash import *
from plot_path import *

## Setup

In [2]:
path = '../data/BTCdata_2018OctDec.csv'
df_oct_dec = pd.read_csv(path)
df_oct_dec = df_oct_dec.dropna()
df_oct_dec['height'].min(), df_oct_dec['height'].max()

(543835, 556458)

In [3]:
path = '../data/BTCdata_2.csv'
df_full = pd.read_csv(path)
df_full = df_full.dropna()

In [4]:
df_full = df_full[df_full['height']<=556458]
df = df_full[543835<=df_full['height']]

In [5]:
df['hashrate'].min(), df['hashrate'].max()

(31.518279011878057, 67.11306695962713)

In [9]:
0.1 / (24*60)

6.944444444444444e-05

In [11]:
H_min = 31.518279011878057
beta = -0.1 / (24*60)
alpha = -beta/H_min
alpha

2.2033069895178425e-06

* Let $H_{min} := 31.52$.
* Assume a positive relation between hash rate and drift
  - more hash rate -> less probability of system's collapse -> the present value of bitcoin goes up
* Let $\mu(t)$ be period-$t$ drift and assume the following version of geometric Brownian motion:
$$
S(t+1) - S(t) = \mu(t) S(t) B(t) + \sigma S(t) \sqrt{B(t)} \epsilon (t),
$$
where $\epsilon(t)$ i.i.d.-drawn from $\mathcal{N}(0,1)$.
* We assume the following affine functional form:
$$
\mu(t) = \alpha H(t) + \beta,
$$
that satisfies
$$
\beta \approx -6.94 \times 10^{-5}, \alpha H_{min} + \beta = 0.
$$
  - $\alpha \approx 2.20 \times 10^{-6}$.
  - $\beta$/min $\approx -0.1$/day
  - The drift is assumed to be (1) zero if the hash rate stays at the same level as the one on the first day of the simulation, and (2) -10%/day if the hash rate is zero.

## Simulation

In [ ]:
# generate epsilons
num_iter = 5000

df_epsilon = pd.DataFrame()
for iter in range(num_iter):
    df_epsilon['iter_{}'.format(iter)] = np.random.normal(size=20000)
df_epsilon.to_csv('/Volumes/Data/research/BDA/simulation/sim_epsilons_{}obs.csv'.format(num_iter), index=False)    

In [5]:
path = '../data/BTCdata_presim.csv'
df = pd.read_csv(path)
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'blocktime': 'block_times', 'price': 'prices', 'probability of success /Eh': 'winning_rates'})

In [6]:
# takes ?? hours
print(datetime.now())
generate_simulation_data(num_iter=5000, prev_data=df)
print(datetime.now())
generate_simulation_data_DAA0(num_iter=5000, prev_data=df)
print(datetime.now())
generate_simulation_data_asert(num_iter=5000, prev_data=df)
print(datetime.now())

2022-02-26 15:25:19.773698
2022-02-26 18:39:54.259987
2022-02-26 20:23:23.718430
2022-02-26 22:09:50.689070


## Block statistics

In [7]:
dir_sim = '/Volumes/Data/research/BDA/simulation/'
filelist = [dir_sim+'hash-price_DAA_asert_blocktime_ps0_5000obs_T=None.csv',
            dir_sim+'hash-price_DAA-0_blocktime_ps0_5000obs.csv',
            dir_sim+'hash-price_DAA-1_blocktime_ps0_5000obs_T=None.csv',
            dir_sim+'hash-price_DAA-2_blocktime_ps0_5000obs_T=None.csv']
df_stats = make_stats(filelist=filelist, dir_sim=dir_sim)

In [8]:
df_stats

,hash-price_DAA_asert_blocktime_ps0_5000obs_T=None.csv,hash-price_DAA-0_blocktime_ps0_5000obs.csv,hash-price_DAA-1_blocktime_ps0_5000obs_T=None.csv,hash-price_DAA-2_blocktime_ps0_5000obs_T=None.csv
mean,9.966584,9.988460,54.159598,10.021586
std,9.998612,9.986182,123.340693,10.106663
over60,1.000000,1.000000,1.000000,1.000000
over120,0.086400,0.066200,0.626400,0.113200
over180,0.000600,0.000400,0.447600,0.000400
